# Modules Importation and File Reading

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt, time

file_location = r"C:\Users\Hp\Downloads\archive\upi_transactions_2024.csv"

In [ ]:
df = pd.read_csv(file_location)
df

# Understanding the Data and Cleaning it

In [ ]:
df.info()
df.describe()
df.nunique()

# Align the dtypes for timestamp

In [ ]:
# Drop any duplicated rows
df2 = df
df2.drop_duplicates(inplace=True)
# No duplicated values in our dataset
# Reformat the dtypes
df2.timestamp = pd.to_datetime(df2.timestamp)
df2.head()

df2.info()
# check if we have the right hour of day when compared to the hour found in the timestamp
df2.spending_hour = df2.timestamp.dt.hour
df2["transaction id"].loc[df2.hour_of_day != df2.spending_hour]
# No error found in hour_of_day in relation to the timestamp

In [ ]:
# Failed Transactions
# Flagged Fraud Transactions
df3 = df2
failed_transactions = df3[df3["transaction_status"] != "SUCCESS"].reset_index(drop=True)
failed_transactions["transaction id"].count()
# 12376 transactions failed
flagged_transactions = df3[df3["fraud_flag"] == 1].reset_index(drop=True)
flagged_transactions["transaction id"].count()
# 480 flagged transactions


# Data Exploration

In [ ]:
# Consumer Analytics: Analyze spending patterns by time, category, and demographics. "Morning, Afternoon, Evening/Night"
morning = time(6, 0, 0)
afternoon = time(12, 0, 0)
evening_night = time(16, 0, 0)


def classify_time(t):
    t = t.time()  # Extract time from datetime
    if morning <= t < afternoon:
        return 'morning'
    elif afternoon <= t < evening_night:
        return 'afternoon'
    else:
        return 'evening/night'


df3["spending_timeclass"] = df3['timestamp'].apply(classify_time)
transaction_count_by_timeclass = df3[['transaction id', 'spending_timeclass']].groupby("spending_timeclass", as_index=False).count()
# df3.columns
transaction_count_by_timeclass.sort_values("transaction id", ascending=False, inplace=True)
sns.barplot(data=transaction_count_by_timeclass, x="spending_timeclass", y="transaction id", hue="spending_timeclass")
plt.ylabel("Transactions Count")
plt.xlabel("Timeclass")
plt.title("Transactions Count by Timeclass")
_ = plt.xticks(rotation=0)

In [ ]:
sum_by_timeclass = df3[['amount (INR)', 'spending_timeclass']].groupby("spending_timeclass").sum().sort_values('amount (INR)', ascending=False)
# df3.columns
sns.barplot(data=sum_by_timeclass, x="spending_timeclass", y="amount (INR)", hue="spending_timeclass")



In [ ]:
df4 = df3[['transaction id', 'amount (INR)', 'spending_timeclass', "merchant_category", 'transaction type']]
shopped_categories_by_timeclass = df4.groupby(['spending_timeclass', "merchant_category"], as_index=False).sum(numeric_only=True).sort_values(["spending_timeclass", "amount (INR)"], ascending=[False, False])
shopped_categories_by_timeclass = shopped_categories_by_timeclass.pivot(index="spending_timeclass", columns="merchant_category", values='amount (INR)')

shopped_categories_by_timeclass

In [ ]:
shopped_categories_by_timeclass.plot(kind="bar")
_ = plt.xticks(rotation=0)
# Shopping tops in the three defined times of day, followed by grocery

In [ ]:
df2.columns

In [ ]:
df5 = df3[['transaction id', 'amount (INR)', 'spending_timeclass', "merchant_category", 'transaction type', "is_weekend"]]
# weekend spending
weekends_data = df5[df5.is_weekend == 1]
weekends_spending = weekends_data[["spending_timeclass", "merchant_category", 'amount (INR)']].groupby(["spending_timeclass", "merchant_category"], as_index=False).sum(numeric_only=True)
weekends_spending.pivot(index="spending_timeclass", columns="merchant_category", values='amount (INR)').plot(kind="bar", figsize=(12, 7), ylabel="Total Spent", xlabel="Time Class")
_ = plt.xticks(rotation=0)


In [ ]:
# monthly spend
df6 = df3
df6.columns
# df6["year"] = df6.timestamp.dt.year
df6["month"] = df6.timestamp.dt.month
df6[['transaction id', 'amount (INR)', 'month']].groupby("month").sum(numeric_only=True).plot(kind="line")

In [ ]:
device_type_usage = df6[["device_type", 'transaction id']].groupby("device_type", as_index=False).count().sort_values("transaction id", ascending=False)
# Most transactions are made through Android devices
# df6.columns
device_type_usage.set_index("device_type").plot(kind="pie", y="transaction id", ylabel="", title="General Phone Type Usage")


In [ ]:
network_type_usage = df6[["device_type", 'transaction id', 'network_type']].groupby("network_type").count().sort_values("transaction id", ascending=False)
network_type_usage.plot(kind="pie", y="transaction id", ylabel="", title="General Network Usage Across All Devices")


In [ ]:
devices_and_networks = df6[['transaction id', 'device_type', 'network_type']].groupby(['device_type', "network_type"], as_index=False).count().sort_values("transaction id", ascending=False)
# devices_and_networks.pivot(index="device_type", values="transaction id", columns="network_type")
merged_df = devices_and_networks.merge(device_type_usage, how='cross', suffixes=("_l", "_r"))
merged_df = merged_df[merged_df["device_type_l"] == merged_df["device_type_r"]]
merged_df["% Usage"] = round((merged_df["transaction id_l"]/merged_df["transaction id_r"])*100, 2)
percentage_device_network_usage = merged_df[["device_type_l", "network_type", "transaction id_l", "% Usage"]].reset_index(drop=True)
percentage_device_network_usage1 = percentage_device_network_usage.sort_values(["device_type_l", "transaction id_l"], ascending=[True, False]).pivot(index="network_type", columns="device_type_l", values="transaction id_l")
percentage_device_network_usage1.plot(kind="bar")
_ = plt.xticks(rotation=0)
percentage_device_network_usage2 = percentage_device_network_usage.sort_values(["device_type_l", "transaction id_l"], ascending=[True, False]).pivot(index="network_type", columns="device_type_l", values="% Usage")
percentage_device_network_usage2.plot(kind="bar")
_ = plt.xticks(rotation=0)